<a href="https://colab.research.google.com/github/Samin-Sadaf7/MasteringObjectDetection/blob/main/MateringObhectDetection_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Bird Mulitple object Detection using FasterRCNN and Tensorflow Object Detection API

Part -2 : training

### Download model from tensorflow 2 model garden

In [2]:
import os
import re #regular expression
import tensorflow as tf
from google.colab import drive
#Mounting to Google Drive
drive.mount('/content/gdrive', force_remount=True)
#Set current directory to MyDrive , put the downloaded files there
%cd gdrive/MyDrive/
# Get current working dir
current_dir= os.getcwd()
print("Root dir:", current_dir)

Mounted at /content/gdrive
/content/gdrive/MyDrive
Root dir: /content/gdrive/MyDrive


In [3]:
#Define some constants
RANDOM_SEED = 99
BATCH_SIZE = 1
NUM_STEPS = 2100
NUM_EVAL_STEPS = 1000

In [4]:
# Check if the directory exists
if not os.path.isdir('faster_rcnn_resnet101_v1_640x640_coco17_tpu-8'):
    # Download the model
    !wget http://download.tensorflow.org/models/object_detection/tf2/20200711/faster_rcnn_resnet101_v1_640x640_coco17_tpu-8.tar.gz
    # Extract the compressed file
    !tar -xf faster_rcnn_resnet101_v1_640x640_coco17_tpu-8.tar.gz
    # Remove the compressed file
    !rm faster_rcnn_resnet101_v1_640x640_coco17_tpu-8.tar.gz

##Generate label map

In [12]:
labels = [
    "Indian Robinrotation",
    "Oriental Magpie Robinrotation",
    "Alexandrine Parakeetrotation",
    "Asian Koelrotation",
    "Indian Paradise-Flycatcherrotation",
    "Laughing Doverotation",
    "Indian Spot-billed Duckrotation",
    "Asian Wooly-necked storkrotation",
    "Bulbulrotation",
    "Barn owlrotation",
    "Woodpeakerrotation",
    "Yellow breasted greenfinchrotation",
    "Red Bearded Bea Eaterrotation",
    "Wrenrotation",
    "Black Kiterotation",
    "Brahminy Kiterotation"
]

with open('label_map.pbtxt', 'w') as f:
    for i, label in enumerate(labels, 1):
        name = label.lower().replace(' ', '_')
        f.write('item {\n')
        f.write('\tname: "{}"\n'.format(name))
        f.write('\tid: {}\n'.format(i))
        f.write('\tdisplay_name: "{}"\n'.format(label))
        f.write('}\n')

### Convert XML annotations to CSV file

In [14]:
!rm xml_to_csv.py
%pwd
#Download the latest version from github
!wget https://raw.githubusercontent.com/ayoubbensakhria/ObjectDetectionFRCNN/master/faster_rcnn_resnet101/xml_to_csv.py

rm: cannot remove 'xml_to_csv.py': No such file or directory
--2024-04-07 23:51:35--  https://raw.githubusercontent.com/ayoubbensakhria/ObjectDetectionFRCNN/master/faster_rcnn_resnet101/xml_to_csv.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1291 (1.3K) [text/plain]
Saving to: ‘xml_to_csv.py’

xml_to_csv.py       100%[===================>]   1.26K  --.-KB/s    in 0s      

2024-04-07 23:51:35 (64.2 MB/s) - ‘xml_to_csv.py’ saved [1291/1291]



In [19]:
#train labels csv
!python xml_to_csv.py --xml_folder $current_dir/birdsod/train --output_csv $current_dir/birdsod/train/train_labels.csv

In [21]:
#test labels csv
!python xml_to_csv.py --xml_folder $current_dir/birdsod/test --output_csv $current_dir/birdsod/test/test_labels.csv

##Convert csv to TFrecords

In [10]:
#Get the latest version , remove the older
!rm generate_tfrecord.py
#Download the latest version from github
!wget https://raw.githubusercontent.com/ayoubbensakhria/ObjectDetectionFRCNN/master/faster_rcnn_resnet101/generate_tfrecord.py

rm: cannot remove 'generate_tfrecord.py': No such file or directory
--2024-04-08 00:30:24--  https://raw.githubusercontent.com/ayoubbensakhria/ObjectDetectionFRCNN/master/faster_rcnn_resnet101/generate_tfrecord.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4117 (4.0K) [text/plain]
Saving to: ‘generate_tfrecord.py’

generate_tfrecord.p 100%[===================>]   4.02K  --.-KB/s    in 0.002s  

2024-04-08 00:30:25 (2.07 MB/s) - ‘generate_tfrecord.py’ saved [4117/4117]



In [13]:
!python generate_tfrecord.py \
  --csv_input=/content/gdrive/MyDrive/birdsod/train/train_labels.csv \
  --image_dir=/content/gdrive/MyDrive/birdsod/train\
  --output_path=/content/gdrive/MyDrive/birdsod/train/train.record

2024-04-08 00:49:27.924204: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Successfully created the TFRecords: /content/gdrive/MyDrive/birdsod/train/train.record


In [14]:
!python generate_tfrecord.py \
  --csv_input=/content/gdrive/MyDrive/birdsod/test/test_labels.csv \
  --image_dir=/content/gdrive/MyDrive/birdsod/test \
  --output_path=/content/gdrive/MyDrive/birdsod/test/test.record

2024-04-08 00:51:09.531669: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Successfully created the TFRecords: /content/gdrive/MyDrive/birdsod/test/test.record
